Walk through of how to run XCP Engine for GRMPY_822831

In [7]:
import flywheel
import datetime
import pytz

fw = flywheel.Client()
proj = fw.lookup('bbl/ALPRAZ_805556')
sessions = proj.sessions()
sessions = [fw.get(x.id) for x in sessions]

ImportError: cannot import name Model

Each session has an `fmriprep` analysis attached (if it was successful). How do we know they were successful?

In [3]:
sessions[0].analyses[0] # check analysis the firts

NameError: name 'sessions' is not defined

In [86]:
sessions[4].files[2].name # check task name for subject 5

'02204_task.zip'

Analyses have attributes such as `state` and `info`:

In [87]:
[(x.gear_info, x.job.state) for x in sessions[0].analyses] # check fmriprep

[({'category': 'analysis',
   'id': '5da887766cbeb000b47d028e',
   'name': 'fmriprep-hpc',
   'version': '0.1.26_1.2.6-1'}, 'cancelled'), ({'category': 'analysis',
   'id': '5da887766cbeb000b47d028e',
   'name': 'fmriprep-hpc',
   'version': '0.1.26_1.2.6-1'}, 'failed'), ({'category': 'analysis',
   'id': '5da887766cbeb000b47d028e',
   'name': 'fmriprep-hpc',
   'version': '0.1.26_1.2.6-1'}, 'failed'), ({'category': 'analysis',
   'id': '5da887766cbeb000b47d028e',
   'name': 'fmriprep-hpc',
   'version': '0.1.26_1.2.6-1'}, 'cancelled'), ({'category': 'analysis',
   'id': '5da886fa6cbeb000b47d0200',
   'name': 'fmriprep',
   'version': '0.1.26_1.2.6-1'}, 'complete'), ({'category': 'analysis',
   'id': '5e0ea8a73a9694007a68028a',
   'name': 'fmriprep-fwheudiconv',
   'version': '0.2.2_1.5.2'}, 'complete'), ({'category': 'analysis',
   'id': '5da887766cbeb000b47d028e',
   'name': 'fmriprep-hpc',
   'version': '0.1.26_1.2.6-1'}, 'complete'), ({'category': 'analysis',
   'id': '5e3c2b176dea

We write a function that loops through the analyses of a session, checks if `fmriprep` ran successfully, and returns the most recent successful `fmriprep` analysis.

In [88]:
def get_latest_fmriprep(session):
    
    if session.analyses:
        
        timezone = pytz.timezone("UTC")
        init_date = datetime.datetime(2018, 1, 1)
        latest_date = timezone.localize(init_date)

        latest_run = None

        for i in session.analyses:
            gear_name = i.gear_info['name']
            state = i.job.state
            date = i.created
            if 'fmriprep' in gear_name and date > latest_date and state =='complete':
                latest_date = date
                latest_run = i
        
        if latest_run is not None:
            fmriprep_out = [x for x in latest_run.files if 'fmriprep' in x.name][0]
            fmriprep_out
            return(fmriprep_out)

        else:
            return None

    else:
        return None

In [89]:
latest_fmriprep = get_latest_fmriprep(sessions[0])
latest_fmriprep

{'classification': {},
 'created': datetime.datetime(2020, 1, 31, 0, 48, 9, 919000, tzinfo=tzutc()),
 'hash': 'v0-sha384-8d21ef7eb30379db741f17eff5a3ecf545f31cf1a346d8867220e358a6fa6d2eaf0e569f8ebe8f1229a16c884fadfd0f',
 'id': '506d460a-7204-40b4-b213-0a6c8d1fc330',
 'info': {},
 'info_exists': None,
 'mimetype': 'application/zip',
 'modality': None,
 'modified': datetime.datetime(2020, 1, 31, 0, 48, 9, 919000, tzinfo=tzutc()),
 'name': 'fmriprep_sub-014665_5e32fccf6dea3102562a7432.zip',
 'origin': {'id': '5e32fccf6dea3102562a7433',
            'method': None,
            'name': None,
            'type': 'job',
            'via': None},
 'replaced': None,
 'size': 1393564803,
 'tags': [],
 'type': 'archive',
 'zip_member_count': None}

In [111]:
#get task file 
def get_taskfile(session):
    for i in range(len(session.files)):
        if session.files[i].name.endswith('task.zip'):
            taskfile=session.files[i]
        else:
            taskfile=None
    return taskfile


In [113]:
d=get_taskfile(sessions[7])
d

{'classification': {},
 'created': datetime.datetime(2020, 2, 6, 15, 32, 9, 243000, tzinfo=tzutc()),
 'hash': 'v0-sha384-ec45f3bf56cdbfbb88e1a4d7da47ea50888dc9264686b3c9bef84f23f989d657ed009149429c5527828df8beee602a6a',
 'id': 'efbbb19e-e2f8-4c40-81ec-16721314bfd9',
 'info': {},
 'info_exists': None,
 'mimetype': 'application/zip',
 'modality': None,
 'modified': datetime.datetime(2020, 2, 6, 15, 32, 8, 860000, tzinfo=tzutc()),
 'name': '02166_task.zip',
 'origin': {'id': 'ttapera@upenn.edu',
            'method': None,
            'name': None,
            'type': 'user',
            'via': None},
 'replaced': None,
 'size': 14395,
 'tags': [],
 'type': 'archive',
 'zip_member_count': None}

Now, we fetch XCP:

In [92]:
xcp = fw.lookup('gears/xcpengine-fw')

In [93]:
xcp # current version 0.0.1_1.00

{'category': 'analysis',
 'created': datetime.datetime(2020, 3, 21, 22, 57, 47, 163000, tzinfo=tzutc()),
 'exchange': {'git_commit': None,
              'rootfs_hash': None,
              'rootfs_url': 'docker://upenn.flywheel.io/xcpengine-fw@sha256:fc2a0eba63e6d572d5f2ebaf97b4087d8189b5a449a93e597bd2fc9d24af021a'},
 'gear': {'author': 'Ted Satterthwaite',
          'cite': 'Ciric, Rastko and Rosen, Adon F. G. and Erus, Guray and '
                  'Cieslak, Matthew and Adebimpe, Azeez and Cook, Philip A. '
                  'and Bassett, Danielle S. and Davatzikos, Christos and Wolf, '
                  'Daniel H. and Satterthwaite, Theodore D., Mitigating head '
                  'motion artifact in functional connectivity MRI',
          'command': None,
          'config': {'analysis_type': {'default': 'xcp',
                                       'description': ' analysis type: '
                                                      '[fc,struc,task,cbf]',
                        

An additional input we need is the design file, which is attached to the project. We use the CBF basil strategy, which is in the name of the design file.

In [94]:
designFile = [x for x in proj.files if "task_acompcor_GSR_alff_reho_fcon" in x.name][0]
designFile = proj.get_file(designFile.name)
designFile

{'classification': {},
 'created': datetime.datetime(2020, 3, 24, 15, 19, 37, 604000, tzinfo=tzutc()),
 'hash': 'v0-sha384-9e7b971e2575dda29f187fe3e42e5f6b0708cb3cf2f6da4d42e43238c54143f22518caff48bfc61dd715de5280e15015',
 'id': '310ea9c5-83e8-443f-ad11-4fa3776f150f',
 'info': {},
 'info_exists': None,
 'mimetype': 'application/octet-stream',
 'modality': None,
 'modified': datetime.datetime(2020, 3, 24, 15, 19, 36, 695000, tzinfo=tzutc()),
 'name': 'task_acompcor_alff_reho_fcon.dsn',
 'origin': {'id': 'larsenb@upenn.edu',
            'method': None,
            'name': None,
            'type': 'user',
            'via': None},
 'replaced': None,
 'size': 3432,
 'tags': [],
 'type': None,
 'zip_member_count': None}

And here's how we run it:

In [8]:
eg = sessions[0]

taskfile = get_taskfile(eg)
fmriprep = get_latest_fmriprep(eg)

eg


NameError: name 'sessions' is not defined

In [97]:
myconfig = {
    'analysis_type': 'task_acompcor_GSR_alff_reho_fcon'
}

myinput = {
    #'antsct': struc,
    'fmriprepdir': fmriprep,
    'designfile': designFile,
    'taskfile': taskfile
    #'img': asl
}
#xcp.run(analysis_label="XCP_SDK_CBF_{}".format(datetime.datetime.now()), destination=eg, inputs=myinput, config=myconfig)

#returns the jobID

In [98]:
eg.label

'002258'

Now we run on each session:

In [115]:
xcp_runs = {}

for i, x in enumerate(sessions):
    ses = fw.get(x.id)

    #struc = get_latest_struct(ses)
    fmriprep = get_latest_fmriprep(ses)
    taskfile1 = get_taskfile(ses)
    
    if  fmriprep and taskfile1 :
        myconfig = {
            'analysis_type': 'task_acompcor_GSR_alff_reho_fcon',
            'task_name' :'emotionid'
        }

        myinput = {
            'fmriprepdir': fmriprep,
            'designfile': designFile,
            'taskfile': taskfile1,
        }
        jobid = xcp.run(analysis_label="XCP_SDK_TASK_GSR{}".format(datetime.datetime.now()), destination=ses, inputs=myinput, config=myconfig)
        xcp_runs[ses.label] = jobid
    else:
        xcp_runs[ses.label] = None

Here are the job IDs:

In [117]:
len(xcp_runs)

96

The reason many didn't run is that 80 is too large a number of volumes for some of the functional ASL scans. Instead, we will redefine the function for ASL and set the floor at 20 and an even number of volumes